In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
import tsfresh as tf
import sklearn

In [16]:
pd.set_option('max_columns', None)

### Data loading

In [24]:
import os
import glob

dataset_path = 'data_feats_90_c/'
csv_files = glob.glob(dataset_path+'*.csv')

all_df = []

for filename in csv_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    all_df.append(df)

df = pd.concat(all_df, axis=0, ignore_index=True)
df.head(1)

,Unnamed: 0,P-PDG__mean,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,"P-PDG__fft_aggregated__aggtype_""centroid""","P-PDG__fft_aggregated__aggtype_""variance""","P-PDG__fft_aggregated__aggtype_""skew""","P-PDG__fft_aggregated__aggtype_""kurtosis""",P-PDG__maximum,P-PDG__minimum,P-PDG__median,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-PDG__friedrich_coefficients__coeff_1__m_3__r_30,P-PDG__friedrich_coefficients__coeff_3__m_3__r_30,P-TPT__mean,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,"P-TPT__fft_aggregated__aggtype_""centroid""","P-TPT__fft_aggregated__aggtype_""variance""","P-TPT__fft_aggregated__aggtype_""skew""","P-TPT__fft_aggregated__aggtype_""kurtosis""",P-TPT__maximum,P-TPT__minimum,P-TPT__median,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.7,P-TPT__quantile__q_0.8,P-TPT__quantile__q_0.9,P-TPT__variation_coefficient,P-TPT__mean_change,P-TPT__mean_second_derivative_central,P-TPT__friedrich_coefficients__coeff_1__m_3__r_30,P-TPT__friedrich_coefficients__coeff_3__m_3__r_30,T-TPT__mean,T-TPT__variance,T-TPT__skewness,T-TPT__kurtosis,"T-TPT__fft_aggregated__aggtype_""centroid""","T-TPT__fft_aggregated__aggtype_""variance""","T-TPT__fft_aggregated__aggtype_""skew""","T-TPT__fft_aggregated__aggtype_""kurtosis""",T-TPT__maximum,T-TPT__minimum,T-TPT__median,T-TPT__quantile__q_0.1,T-TPT__quantile__q_0.2,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__variation_coefficient,T-TPT__mean_change,T-TPT__mean_second_derivative_central,T-TPT__friedrich_coefficients__coeff_1__m_3__r_30,T-TPT__friedrich_coefficients__coeff_3__m_3__r_30,P-MON-CKP__mean,P-MON-CKP__variance,P-MON-CKP__skewness,P-MON-CKP__kurtosis,"P-MON-CKP__fft_aggregated__aggtype_""centroid""","P-MON-CKP__fft_aggregated__aggtype_""variance""","P-MON-CKP__fft_aggregated__aggtype_""skew""","P-MON-CKP__fft_aggregated__aggtype_""kurtosis""",P-MON-CKP__maximum,P-MON-CKP__minimum,P-MON-CKP__median,P-MON-CKP__quantile__q_0.1,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,P-MON-CKP__quantile__q_0.4,P-MON-CKP__quantile__q_0.6,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.8,P-MON-CKP__quantile__q_0.9,P-MON-CKP__variation_coefficient,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,P-MON-CKP__friedrich_coefficients__coeff_1__m_3__r_30,P-MON-CKP__friedrich_coefficients__coeff_3__m_3__r_30,T-JUS-CKP__mean,T-JUS-CKP__variance,T-JUS-CKP__skewness,T-JUS-CKP__kurtosis,"T-JUS-CKP__fft_aggregated__aggtype_""centroid""","T-JUS-CKP__fft_aggregated__aggtype_""variance""","T-JUS-CKP__fft_aggregated__aggtype_""skew""","T-JUS-CKP__fft_aggregated__aggtype_""kurtosis""",T-JUS-CKP__maximum,T-JUS-CKP__minimum,T-JUS-CKP__median,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.8,T-JUS-CKP__quantile__q_0.9,T-JUS-CKP__variation_coefficient,T-JUS-CKP__mean_change,T-JUS-CKP__mean_second_derivative_central,T-JUS-CKP__friedrich_coefficients__coeff_1__m_3__r_30,T-JUS-CKP__friedrich_coefficients__coeff_3__m_3__r_30,QGL__mean,QGL__variance,QGL__skewness,QGL__kurtosis,"QGL__fft_aggregated__aggtype_""centroid""","QGL__fft_aggregated__aggtype_""variance""","QGL__fft_aggregated__aggtype_""skew""","QGL__fft_aggregated__aggtype_""kurtosis""",QGL__maximum,QGL__minimum,QGL__median,QGL__quantile__q_0.1,QGL__quantile__q_0.2,QGL__quantile__q_0.3,QGL__quantile__q_0.4,QGL__quantile__q_0.6,QGL__quantile__q_0.7,QGL__quantile__q_0.8,QGL__quantile__q_0.9,QGL__variation_coefficient,QGL__mean_change,QGL__mean_second_derivative_central,QGL__friedrich_c

In [25]:
df.drop(['Unnamed: 0'], axis='columns', inplace=True)      #leftover index column, unnecessary

df = df[df.class_code != 7]                                #paper does not use event class 7
df['class_code'].replace({8: 7}, inplace=True)


### Handling NA values

In [26]:
def print_na_sum(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        nans = df.isna().sum(axis=0)
        print(nans[nans!=0])

print_na_sum(df)

P-PDG__fft_aggregated__aggtype_"centroid"                 8571
P-PDG__fft_aggregated__aggtype_"variance"                 8571
P-PDG__fft_aggregated__aggtype_"skew"                    37244
P-PDG__fft_aggregated__aggtype_"kurtosis"                37244
P-PDG__variation_coefficient                              8571
P-PDG__friedrich_coefficients__coeff_1__m_3__r_30        20675
P-PDG__friedrich_coefficients__coeff_3__m_3__r_30        20675
P-TPT__fft_aggregated__aggtype_"centroid"                   64
P-TPT__fft_aggregated__aggtype_"variance"                   64
P-TPT__fft_aggregated__aggtype_"skew"                    30450
P-TPT__fft_aggregated__aggtype_"kurtosis"                30450
P-TPT__variation_coefficient                                64
P-TPT__friedrich_coefficients__coeff_1__m_3__r_30        12623
P-TPT__friedrich_coefficients__coeff_3__m_3__r_30        12623
T-TPT__fft_aggregated__aggtype_"centroid"                 6514
T-TPT__fft_aggregated__aggtype_"variance"              


Removing features with too many NA values

In [27]:
df.drop(['QGL__variation_coefficient'], axis='columns', inplace=True)
df.drop(list(df.filter(regex = 'friedrich')), axis = 1, inplace = True)
df.drop(list(df.filter(regex = 'fft_aggregated')), axis = 1, inplace = True)

In [28]:
print_na_sum(df)

P-PDG__variation_coefficient        8571
P-TPT__variation_coefficient          64
T-TPT__variation_coefficient        6514
P-MON-CKP__variation_coefficient    1200
T-JUS-CKP__variation_coefficient    1819
dtype: int64


Imputing leftover na values, for each class separately

In [31]:
imputed_df_list = []
for i in df['class_code'].unique():
    query = 'class_code == ' + str(i)
    imputed = tf.utilities.dataframe_functions.impute(df.query(query))
    imputed_df_list.append(imputed)
imputed_df = pd.concat(imputed_df_list)

/home/marko/.local/lib/python3.8/site-packages/pandas/core/generic.py:9032: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(cond, other, inplace, axis, level, errors=errors)


In [32]:
print_na_sum(imputed_df)

Series([], dtype: int64)
